In [1]:
import tensorflow.keras as keras
import keras.applications.resnet50 as resnet50
from tensorflow.keras import backend

Using TensorFlow backend.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Flatten,ZeroPadding2D, Add
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Activation
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [3]:
import tensorflow.keras as keras

In [4]:
def conv_block(input_data,units):
    units1, units2, units3 = units 
    x=Conv2D(units1,(1,1),padding='same')(input_data)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2D(units2,(3,3),padding='same')(x)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Conv2D(units3,(1,1),padding='same')(x)
    x=BatchNormalization()(x)
    shortcut = Conv2D(units3, (1, 1))(input_data)
    shortcut = BatchNormalization()(shortcut)
    x=Add()([x,shortcut ])
    return x

In [5]:
def identity_block(input_data,units):
    units1, units2, units3 = units 
    x=keras.layers.Conv2D(units1,(1,1),padding='same')(input_data)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.Conv2D(units2,(3,3),padding='same')(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.Conv2D(units3,(3,3),padding='same')(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.Add()([x, input_data])
    
    return x

In [6]:
def buildresnet():
    inputs= keras.Input(shape=(320,480,6))
    x=keras.layers.ZeroPadding2D(padding=(1,1))(inputs)
    x=keras.layers.Conv2D(64,(7,7),padding='same')(x)
    x=keras.layers.BatchNormalization()(x)
    x=keras.layers.Activation('relu')(x)
    x=keras.layers.ZeroPadding2D(padding=(1,1))(x)
    x=keras.layers.MaxPooling2D(3,padding='same')(x)

    x=conv_block(x,[64, 64, 256])
    
    for i in range(0,2):
        x=identity_block(x,[64, 64, 256])
        
    x=conv_block(x,[128, 128, 512])
    
    for i in range(0,3):
        x=identity_block(x,[128, 128, 512])
        
    x=conv_block(x,[256, 256, 1024])
    
    for i in range(0,5):
        x=identity_block(x,[256, 256, 1024])
        
    x=conv_block(x,[512, 512, 2048])
    
    for i in range(0,2):
        x=identity_block(x,[512, 512, 2048])
        
    x =keras.layers.GlobalAveragePooling2D()(x)
    
    outputs =keras.layers.Dense(10, activation='softmax')(x)
    resnet = keras.Model(inputs, outputs)
    
    return resnet

In [7]:
resnet = buildresnet()

In [8]:
resnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 480, 6) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 322, 482, 6)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 322, 482, 64) 18880       zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 322, 482, 64) 256         conv2d[0][0]                     
______________________________________________________________________________________________

In [9]:
baseModel = keras.applications.ResNet50(weights="imagenet")

In [10]:
baseModel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [38]:
weights = baseModel.layers[3].get_weights()

In [39]:
print(weights)

[array([1.0913312 , 1.1138613 , 0.87221205, 1.3775853 , 1.1671193 ,
       1.1966964 , 0.91123503, 1.5656891 , 2.5536506 , 0.8813567 ,
       1.1953614 , 0.86340654, 0.9432478 , 2.4865716 , 1.0368608 ,
       1.2843996 , 1.4514154 , 1.3008332 , 1.0089829 , 1.3758818 ,
       1.0634737 , 1.1025162 , 2.6344054 , 1.0763955 , 1.0826234 ,
       1.4103851 , 1.982566  , 0.744998  , 0.9306326 , 2.0845473 ,
       1.4146942 , 0.879981  , 0.8721337 , 0.5126429 , 1.2891649 ,
       0.90243804, 0.8805454 , 1.864482  , 1.380977  , 1.01026   ,
       1.3941606 , 1.040888  , 1.2940407 , 0.5283866 , 0.9631836 ,
       2.6685867 , 1.1683974 , 1.1092062 , 0.8321394 , 1.0036737 ,
       1.4683139 , 1.5917623 , 0.55286855, 1.2541881 , 1.5560241 ,
       1.069061  , 0.8508632 , 0.92903084, 1.411313  , 0.9452564 ,
       0.8614338 , 0.9161029 , 1.0425918 , 1.01351   ], dtype=float32), array([ 1.4794971 , -1.1218344 , -1.064731  , -1.8752855 ,  1.3335849 ,
        2.0549939 , -1.1351001 , -0.95681113, -2.65

In [33]:
for i in range(3,17):
    
    weights = baseModel.layers[i].get_weights()
    resnet.layers[i].set_weights(weights)
    

for layer in resnet.layers: 
    print(layer.get_config(), layer.get_weights())

In [40]:
res_weights = resnet.layers[3].get_weights()
print(res_weights)

[array([1.0913312 , 1.1138613 , 0.87221205, 1.3775853 , 1.1671193 ,
       1.1966964 , 0.91123503, 1.5656891 , 2.5536506 , 0.8813567 ,
       1.1953614 , 0.86340654, 0.9432478 , 2.4865716 , 1.0368608 ,
       1.2843996 , 1.4514154 , 1.3008332 , 1.0089829 , 1.3758818 ,
       1.0634737 , 1.1025162 , 2.6344054 , 1.0763955 , 1.0826234 ,
       1.4103851 , 1.982566  , 0.744998  , 0.9306326 , 2.0845473 ,
       1.4146942 , 0.879981  , 0.8721337 , 0.5126429 , 1.2891649 ,
       0.90243804, 0.8805454 , 1.864482  , 1.380977  , 1.01026   ,
       1.3941606 , 1.040888  , 1.2940407 , 0.5283866 , 0.9631836 ,
       2.6685867 , 1.1683974 , 1.1092062 , 0.8321394 , 1.0036737 ,
       1.4683139 , 1.5917623 , 0.55286855, 1.2541881 , 1.5560241 ,
       1.069061  , 0.8508632 , 0.92903084, 1.411313  , 0.9452564 ,
       0.8614338 , 0.9161029 , 1.0425918 , 1.01351   ], dtype=float32), array([ 1.4794971 , -1.1218344 , -1.064731  , -1.8752855 ,  1.3335849 ,
        2.0549939 , -1.1351001 , -0.95681113, -2.65